
# 

#  CuPy Lab 1
---
Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvidia-smi



### Learning Objectives
- **The goal of this lab is to:**
     - quickly get you started with CuPy from beginner to intermediate level
     - teach you application of GPU programming concept to HPC field(s)
     - show you how to maximize the throughput of your HPC implementation through computational speedup on the GPU.      


##  What is CuPy?
- CuPy is an implementation of NumPy-compatible multi-dimensional array on CUDA
- CuPy consists of :
    - cupy.ndarray 
    - the core multi-dimensional array class 
    - many functions 
- It supports a subset of numpy.ndarray interface which include:
    - Basic and advance indexing 
    - Data types (int32, float32, uint64, complex64,... )
    - Array manipulation routine (reshape)
    - Linear Algebra functions (dot, matmul, etc)
    - Reduction along axis (max, sum, argmax, etc)

In [ ]:
import numpy as np
X = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#Basic indexing and slicing
print(X[5:])

#output: [5 6 7 8 9]

print(X[1:7:2])

#output: [1 3 5]

#reduction and Linear Algebra function
max(X)
#output:  9

In [ ]:
import numpy as np
#Advance indexing
X = np.array([[1, 2],[3, 4],[5, 6]])
print(X[[0, 1, 2], [0, 1, 0]])

#output: [1 4 5]

B = np.array([1,2,3,4], dtype=np.float32)
C = np.array([5,6,7,8], dtype=np.float32)
print("matmul:",np.matmul(B, C))

#output: matmul: 70.0

In [ ]:
import numpy as np
#data type and array manipulation routine 
A =1j*np.arange(9, dtype=np.complex64).reshape(3,3)
print(A)


#output:
#[[0.+0.j 0.+1.j 0.+2.j]
# [0.+3.j 0.+4.j 0.+5.j]
# [0.+6.j 0.+7.j 0.+8.j]]

##  Features of CuPy

- **Features of CuPy includes:**
    - User-define elementwise CUDA kernels
    - User-define reduction CUDA kernels
    - Fusing CUDA kernels to optimize user-define calculation
    - Customizable memory allocator and memory pool
    - cuDNN utilities
- These features  are developed to support performance
- CuPy uses on-the-fly kernel synthesis: when a kernel call is required, it compiles a kernel code optimized for the shapes and dtypes of given arguments, sends it to the GPU device, and executes the kernel.
- CuPy also caches the kernel code sent to GPU device within the process, which reduces the kernel transfer time on further calls. 


###  CuPy Architecture

The CuPy Architecture expose functionalities within CuPy API that allows developers (or users) to create a User-defined CuDA Kernel, make use of DNN utility through the cuDNN functionality, solve Linear algebra through cuTENSOR, cuBLAS, cuSOLVER, and cuSPARSE api functions. Also, Random numbers can be generated using cuRAND. Sort, Scan and Reduction operations are conveniently executed through the use of CUB and Thrust, and lastly, Multi-GPU data transfer task are initiated through the use of NCCL functionality. It important to know that all the aformentioned api functionalities rely on CUDA, while the CUDA itself runs 
on the NVIDIA GPU as shown in the diagram below. 

<img src="../images/cupy_arch.png">


## CuPy Fundamentals
- **Cupy.ndarray**: CuPy is a GPU array backend that implements a subset of NumPy interface.
```python
#CuPy version
import cupy as cp
X_gpu = cp.array([1, 2, 3, 4, 5])
#NumPy version
import numpy as np
x = np.array([1, 2, 3, 4, 5])
```

- CuPy considers the current device as the default device with device ID 0. It also allows temporary switch between GPU devices.

```python
import cupy as cp
####### Current device (GPU ID: 0)##############
gpu_0 = cp.array([1, 2, 3, 4, 5])

# Switch device
cp.cuda.Device(1).use()
gpu_1 = cp.array([1, 2, 3, 4])

###### Switch GPU temporarily################
import numpy as np
with cp.cuda.Device(1):
      gpu_1 = cp.array([1, 2, 3, 4])
# back to device id 0
gpu0 = cp.array([1, 2, 3, 4, 5]) 
```

### Data transfer
- Arrays can be moved from Host to Device (CPU -> GPU) using **cupy.asarray**

In [ ]:
import cupy as cp
import numpy as np
x = np.array([1, 2, 3, 4, 5])
x_gpu = cp.asarray(x)
print(x_gpu)

#output: [1 2 3 4 5]

- Device array can be move to Host(GPU -> CPU) using: **cupy.asnumpy or cupy.ndarray.get()**


In [ ]:
import cupy as cp
import numpy as np
x_gpu = cp.array([1, 2, 3, 4, 5])
#copy to Host
x_cpu = cp.asnumpy(x_gpu)
print("x_cpu: ",x_cpu)

#alternative option
x_cpu_alt = x_gpu.get()
print("x_cpu_alt: ",x_cpu_alt) 

#output: 
#x_cpu:  [1 2 3 4 5]
#x_cpu_alt:  [1 2 3 4 5]

- In order to transfer an array between devices(GPU to GPU), **cupy.ndarray** is used.

In [ ]:
import cupy as cp
with cp.cuda.Device(0):
    x_gpu_0 = cp.ndarray([ 2, 3, 3]) 
print("x_gpu_0:\n", x_gpu_0)

with cp.cuda.Device(0):
      x_gpu_1 = cp.asarray(x_gpu_0)
print("x_gpu_1:\n", x_gpu_1)

#output
#x_gpu_0:
# [[[0. 0. 0.]
#  [0. 0. 0.]
#  [0. 0. 0.]]

# [[0. 0. 0.]
#  [0. 0. 0.]
#  [0. 0. 0.]]]


### GPU  & CPU agnostic code
- The compatibility of CuPy with NumPy enables the implementation of CPU/GPU generic code using **cupy.get_array_module()**

In [ ]:
import cupy as cp
import numpy as np

#example: log(1 + exp(x))
x_cpu  = np.array([1, 2, 3, 4, 5])
x_gpu  = cp.get_array_module(x_cpu)
result = x_gpu.maximum(0, x_cpu) + x_gpu.log1p(x_gpu.exp(-abs(x_cpu)))
print(result)

#output: [1.31326169 2.12692801 3.04858735 4.01814993 5.00671535]

#An explicit conversion to a host 
x_gpu  = cp.array([6, 7, 8, 9, 10])
result = cp.asnumpy(x_gpu) + x_cpu
print(result)

#output: [ 7  9 11 13 15]

#An explicit conversion to a device
result = x_gpu + cp.asarray(x_cpu)
print(result)

#output: [ 7  9 11 13 15]


##  CUDA Kernels

- **CUDA Kernels can be define in Cupy as follows:**

    - Elementwise Kernels
    - Reduction Kernels
    - Raw Kernels
    - Kernel Fusion
- These kernels are user-defined based.

### Elementwise Kernels
- The ElementwiseKernel class is used to define this type of kernel.
- This kernel consists of four parts which includes:
    1. a list of input argument 
    2. a list of output argument
    3. a loop body code
    4. kernel name
- Variable name starting with underscore “_” , “n”, and “i” are regarded as reserved keywords.

#### Example: z = x*w + b

In [ ]:
import cupy as cp

input_list = 'float32 x , float32 w, float32 b'
output_list = 'float32 z'
code_body  = 'z =  (x * w) + b'

# elementwisekernel class defined
dnnLayerNode = cp.ElementwiseKernel(input_list, output_list, code_body,'dnnLayerNode')

# data
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
b = cp.array([-0.5], dtype=cp.float32)
z = cp.empty((3,3), dtype=cp.float32)

# kernel call with argument passing
dnnLayerNode(x,w,b,z)
print(z)

#output:
#[[-0.5  0.5  3.5]
# [ 8.5 15.5 24.5]
# [35.5 48.5 63.5]]


### Elementwise Kernel: Generic-type kernels
- It can be used to define a generic-type kernels. It treats a type specifier of one character as a type placeholder.

In [ ]:
import cupy as cp

input_list = 'T x , T w, T b'
output_list = 'T z'
code_body  = 'z =  (x * w) + b'

# elementwisekernel class defined
dnnLayerNode = cp.ElementwiseKernel(input_list, output_list, code_body,'dnnLayerNode')
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
b = cp.array([-0.5], dtype=cp.float32)
z = cp.empty((3,3), dtype=cp.float32)

# kernel call with argument passing
dnnLayerNode(x,w,b,z)
print(z)

#output:
#[[-0.5  0.5  3.5]
# [ 8.5 15.5 24.5]
# [35.5 48.5 63.5]]

### Reduction Kernels

- Reduction kernel is implemented through the ReductionKernel class. 
- In order to implement this kernel class, the following parts must be defined:
    - **Identity value**: to initialize reduction value.
    - **Mapping expression**: Used for the pre-processing of each element to be reduced.
    - **Reduction expression**: It is an operator to reduce the multiple mapped values. The special variables **a** and **b** are used for its operands.
    - **Post mapping expression**: It is used to transform the resulting reduced values. The special variable a is used as its input. Output should be written to the output parameter.


**Example:  𝑧=∑_(𝑖=1)𝑥_𝑖 *𝑤_𝑖+𝑏**

In [ ]:
import cupy as cp
dnnLayer = cp.ReductionKernel(
	'T x, T w, T bias',              
	'T z',                         
	'x * w',                      
	'a + b', 
	'z = a + bias',              
	'0',                            
	'dnnLayer'  )
x = cp.arange(10, dtype=np.float32).reshape(2,5)
w = cp.arange(10, dtype=np.float32).reshape(2,5)
bias = -0.1
z = dnnLayer(x,w,bias)
print(z)

#output: 284.9 

## Raw Kernels

- Raw kernels enables  the  direct use of kernels from CUDA source, and it is defined through the RawKernel class.
- The RawKernel object allows you to call the kernel with CUDA’s cuLaunchKernel interface. In other words, you have control over:
    - grid size
    - block size
    - shared memory size 
    - and stream. 
<img src="../images/cupy_kernel_memory.png">

In [ ]:
import cupy as cp
add_kernel = cp.RawKernel(r'''
extern "C" __global__
void add_func(const float* x1, const float* x2, float* y) {
int tid = blockDim.x * blockIdx.x + threadIdx.x;
y[tid] = x1[tid] + x2[tid];
}
''', 'add_func')

N = 100
shape = (10, 10)

x1 = cp.arange(N, dtype=cp.float32).reshape(shape)
x2 = cp.arange(N, dtype=cp.float32).reshape(shape)
y = cp.zeros((shape), dtype=cp.float32)

add_kernel((10,), (10,), (x1, x2, y)) 

print(y)

#output:
#[[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.]
#[ 20.  22.  24.  26.  28.  30.  32.  34.  36.  38.]
#[ 40.  42.  44.  46.  48.  50.  52.  54.  56.  58.]
#[ 60.  62.  64.  66.  68.  70.  72.  74.  76.  78.]
#[ 80.  82.  84.  86.  88.  90.  92.  94.  96.  98.]
#[100. 102. 104. 106. 108. 110. 112. 114. 116. 118.]
#[120. 122. 124. 126. 128. 130. 132. 134. 136. 138.]
#[140. 142. 144. 146. 148. 150. 152. 154. 156. 158.]
#[160. 162. 164. 166. 168. 170. 172. 174. 176. 178.]
#[180. 182. 184. 186. 188. 190. 192. 194. 196. 198.]]


<img src="../images/raw_kernel.png">

#### Raw Kernels : Complex-value arrays

In [ ]:
import cupy as cp

complex_kernel = cp.RawKernel(r'''
#include <cupy/complex.cuh>
extern "C" __global__
void my_func(const complex<float>* x1, const complex<float>* x2, complex<float>* y, float a){
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    y[tid] = x1[tid] + a * x2[tid];
}
''', 'my_func')

x1 = cp.arange(25, dtype=cp.complex64).reshape(5,5)
x2 = 1j*cp.arange(25, dtype=cp.complex64).reshape(5,5)
y = cp.zeros((5,5), dtype=cp.complex64)

complex_kernel((1,),(25,),(x1, x2,y,cp.float32(2.0)))
print(y)

#output:
#[[ 0. +0.j  1. +2.j  2. +4.j  3. +6.j  4. +8.j]
#[ 5.+10.j  6.+12.j  7.+14.j  8.+16.j  9.+18.j]
#[10.+20.j 11.+22.j 12.+24.j 13.+26.j 14.+28.j]
#[15.+30.j 16.+32.j 17.+34.j 18.+36.j 19.+38.j]
#[20.+40.j 21.+42.j 22.+44.j 23.+46.j 24.+48.j]]


In [ ]:
#This also produced the same output:
complex_kernel((5,), (5,), (x1, x2, y, cp.float32(2.0)))
print(y)

In [ ]:
####### Kernel Attributes ##################
print("max_dynamic_shared_size_bytes: ", complex_kernel.max_dynamic_shared_size_bytes )

print("max_threads_per_block: ", complex_kernel.max_threads_per_block )

print("attributes: ",complex_kernel.attributes)


### Raw Modules 

- The **RawModule** class is used to defining a large raw CUDA C source or loading an existing CUDA binary.
- It is initialized by a CUDA C source code having a several kernels (functions) such that needed kernels are retrieved by calling the **get_function()** method.

```python
import cupy as cp
loaded_from_source = r'''
extern "C" {
__global__ void test_sum(const float* A, const float* B, float* C, int N)
 { 
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if(tid < N)
    {
      C[tid] = A[tid] + B[tid]; 
    }
 }
 __global__ void test_multiply(const float* A, const float* B, float* C, int N )
 {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if(tid < N)
    {
        C[tid] = A[tid] * B[tid];
    }
 }
}'''
```
##### Example:

In [ ]:
import cupy as cp

load_raw_module = r'''
extern "C" {
__global__ void sum_ker(const float* a, const float* b, float* c)
 { 
    unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
    c[tid] = a[tid] + b[tid]; 
    
 }
 __global__ void multiply_ker(const float* a, const float* b, float* c )
 {
    unsigned int tid = blockDim.x * blockIdx.x + threadIdx.x;
    c[tid] = a[tid] * b[tid];
 }
}'''

module = cp.RawModule(code = load_raw_module)

ker_sum = module.get_function('sum_ker')
ker_times = module.get_function('multiply_ker')

a = cp.arange(25, dtype=cp.float32).reshape(5,5)
b = cp.ones((5,5), dtype=cp.float32)
c = cp.zeros((5,5), dtype=cp.float32)

In [ ]:
# run the above cell before runing this cell
ker_sum((1,),(25,), (a,b,c))
print(c)

#output:
#[[ 1.  2.  3.  4.  5.]
#[ 6.  7.  8.  9. 10.]
#[11. 12. 13. 14. 15.]
#[16. 17. 18. 19. 20.]
#[21. 22. 23. 24. 25.]]

In [ ]:
ker_times((5,),(5,),(a,b,c))
print(c)


### Kernel Fusion

- Kernel fusion is a decorator that fuses functions. It can be used to define an elementwise or reduction kernels easily.


In [ ]:
import cupy as cp

@cp.fuse(kernel_name='dnnlayerNode')
def dnnlayerNode(x, w, bias):
    return  (x * w) + bias
    
x = cp.arange(9, dtype=cp.float32).reshape(3,3)
w = cp.arange(9, dtype=cp.float32).reshape(3,3)
bias = cp.array([-0.5], dtype=cp.float32)
         
z = dnnlayerNode(x,w,bias)
print(z)

#output:
#[[-0.5  0.5  3.5]
#[ 8.5 15.5 24.5]
#[35.5 48.5 63.5]]

## Summary
<img src="../images/cupy_summary.png" width="80%" height="80%">

---
## Lab Task

In this section, you are expected to click on the **Serial code Lab Assignment** link and proceed to Lab 2. In this lab you will find three python serial code functions. You are required to revise the **pair_gpu** function and make it run on the GPU, and likewise do a few modifications on the **main** function.

## <div style="text-align:center; color:#FF0000; border:3px solid red;height:80px;"> <b><br/> [Serial Code Lab Assignment](serial_RDF.ipynb) </b> </div>

 
---


## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *


**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip).

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

---

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start_python.ipynb>HOME</a></p>

---

# Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---


##  References
- https://docs.cupy.dev/en/stable/
- https://cupy.dev/
- CuPy Documentation Release 8.5.0, Preferred Networks, inc. and Preferred Infrastructure inc., Feb 26, 2021.
- Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018.
- Crissman Loomis and Emilio Castillo, CuPy Overview: NumPy Syntax Computation with Advanced CUDA Features, GTC Digital March, March 2020.
- https://www.gpuhackathons.org/technical-resources
- https://rapids.ai/start.html

--- 

## Licensing 

This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).